# A Practical Guide to Computing Diffusivity of Amorphous Materials Using VASP

This guide provides a detailed procedure with all necessary scripts to compute diffusivity of any amorphous material using the VASP package.
For scientific background, see [Angew. Chem. Int. Ed. 2024, 63, e202315628](https://onlinelibrary.wiley.com/doi/full/10.1002/anie.202315628), and [ACS Appl. Mater. Interfaces 2020, 12, 35748](https://pubs.acs.org/doi/10.1021/acsami.0c10000). We will use $LiTaCl_6$ in [Angew. Chem. Int. Ed. 2024, 63, e202315628](https://onlinelibrary.wiley.com/doi/full/10.1002/anie.202315628) as the material, combined with methods from [ACS Appl. Mater. Interfaces 2020, 12, 35748](https://pubs.acs.org/doi/10.1021/acsami.0c10000) to achieve efficient calculation of diffusivity.

# 0. Preparation

First of all, we need to install some packages. 

Download and compile [packmol](https://m3g.github.io/packmol/). 

The code was tested in a conda environment and that's the recommended way to use the code. Suppose you have conda installed/loaded, do the following to create an environment for this project:

<code>conda create --name vaspenv python numpy scipy ase pymatgen</code>. 

Then type <code>conda activate vaspenv</code> to use the environment.

# 1. Equilibration in Liquid Phase
The first step is to melt and equilibrate the material above its melting point but below its boiling point. We follow [ACS Appl. Mater. Interfaces 2020, 12, 35748](https://pubs.acs.org/doi/10.1021/acsami.0c10000) to create a randomly packed initial structure using the packmol package.

<code>cd initialStructure</code> 

Modify the user input section of <code>getInitialStructure.py</code> for your own system.

```python
# ----- user data -------------------------------------------------------------
molar_masses = {"Li": 6.9410, "Ta": 180.94788, "Cl": 35.45}     # g mol⁻¹
stoich       = {"Li": 1, "Ta": 1, "Cl": 6}                      # LiTaCl₆
density      = 2.96                      # g cm⁻³
box_diam     = 12.0                      # Å
packmolPath = "/pscratch/sd/x/xlliu9/Software/packmol/packmol"
# ---------------------------------------------------------------------------
```
The user needs to supply molar masses and stoichiometry for each element in the system, as well as density, diameter of the cubic box, and the location of packmol package. Note the code will create a cubic box with diameters slightly greater than <code>box_diam</code>.
Run the code and you will get a <code>POSCAR</code> file in the same directory. This is the starting geometry for our simulations.

We then equilibrate the structure at a suitable temperature. For $LiTaCl_6$, we choose 1500 K. Copy the POSCAR file to <code>eql_1500</code> and prepare other files. To better reach equilibrium, a Langevin thermostat should be used. Run <code>pltAll.py</code> frequently to monitor the status of MD. Modify the user input section of <code>pltAll.py</code> for each directory you want to monitor:

```python
# ----- user data -------------------------------------------------------------
timestep = 1.0 # in fs
timeAvg = 2 # in ps
maxRestart = 10

last_n_ps_steps = int(timeAvg * 1e3 / timestep)
dirs = ["eql_1500"]
# ---------------------------------------------------------------------------   
```
The code plots potential energy, temperature, pressure and volume in dirs and restart1/ up to restart10/ directories within each of them, and saves a png file named using the directories. A green curve showing the running average property of <code>timeAvg</code> is also plotted. It is considered to have reached equilibrium if the difference of average energy between last 2 ps and last 4 ps is within 1meV/atom.

# 2. Iteratively Optimizing Density of the Amorphous Phase

Once it reached equilibrium, we need to determine its density at 1500 K. We do so by iteratively scale the lattice parameters according to:

$\frac{\Delta V}{V} \approx -\frac{\Delta P}{K}$, 

where $V$ is the volume of the cell, $P$ is pressure, and $K$ is bulk modulus. We approximate $K$ by $100GPa$.

Copy the equilibrated <code>CONTCAR</code> to <code>scale/POSCAR</code>. Copy other input files and modify <code>iterativeScale.sh</code> for your own project. The script runs AIMD for 4 ps at most 10 times and takes the average pressure in the last 2 ps as input to scale the lattice until the pressure drops below 5 kB.

# 3. Fast Quench to Room Temperature
Having obtained a good liquid structure, we then quench the material down to room temperature. 

Copy the <code>POSCAR</code> from <code>scale/</code> to <code>quench/</code>

The <code>quench/quench.sh</code> script quenches the material from 1500 K to 300 K in a decrement of 300 K every 2 ps. Modify it for your own project.

# 4. Optimization of Density at Room Temperature

Having quenched down to 300 K, we then quench the material down to 0 K to optimize for more accurate density at room temperature.

Copy <code>quench/quench_300/CONTCAR</code> to <code>opt/</code>

# 5. Training Machine Learning Force Field around Room Temperature

# 6. Running the MLFF to Determine Diffusivities and Activation Energy